# Introduction
In this notebook we examine the idea of a hash function and what this means for proof-of-work. In particular we focus on how to make something that is difficult to compute and yet easy to verify for the purposes of a blockchain

## Hash Functions / Signatures
Here we start out with hash functions where we use the [SHA256](https://en.wikipedia.org/wiki/SHA-2) algorithm built into python. We wrap the function up a bit so we can use it with both strings and numbers.

In [ ]:
from hashlib import sha256 # a 256-bit hash function 
def simple_hash_func(in_value):
    return sha256('{}'.format(in_value).encode()).hexdigest()[-7:]

Using the hash function, here we can use the hash function to encode a _signature_ or fingerprint of the value. The primary purpose of this _signature_ is that it encapsulates the value without actually containing it. Individual hashes are very quick to compute which is important for making a blockchain easy to verify.

In [ ]:
%%time
my_favorite_number = 12345
print(my_favorite_number,'\t',simple_hash_func(my_favorite_number))
# we can now try adding one to this number and we see that the hash changes
print(my_favorite_number+1,'\t',simple_hash_func(my_favorite_number+1))
if simple_hash_func(my_favorite_number) != simple_hash_func(my_favorite_number+1):
    print('Hashes do not match!')

## List of transactions
Following the slides we can do the same task with a list of transactions
```
Tim pays Joel $3
Joel pays Kevin $1.5
Tim pays Joel $3
```


In [ ]:
transactions = [
    ['2017-12-12', 'Tim pays Joel $3'],
    ['2017-12-13', 'Joel pays Kevin $1.5'],
    ['2017-12-13', 'Tim pays Joel $3']
]
last_signature = ''
signed_transactions = []
for date, content in transactions:
    last_signature = simple_hash_func(date+content+last_signature)
    signed_transactions += [[date, content, last_signature]]
                       
for date, content, signature in signed_transactions:
    print('\t'.join([date, content, signature]))

We can check to see if it is really signed by trying to change the amount Joel pays Kevin to $15

In [ ]:
transactions[1][1] = 'Joel pays Kevin $15.'
last_signature = ''
new_signed_transactions = []
for date, content in transactions:
    last_signature = simple_hash_func(date+content+last_signature)
    new_signed_transactions += [[date, content, last_signature]]
                       
for (date, content, new_signature), (_, _, old_signature) in zip(new_signed_transactions, signed_transactions):
    print('\t'.join([date, content]))
    print('\t\t\tnew: ', new_signature)
    print('\t\t\told: ', old_signature)

* We see that it worked, even though we only changed one character (moved the period)

## Proof of Work
The challenge of blockchain is that we need something that is **difficult to compute** and easy to verify. This assymetry is what makes the problem more challenging. Here we introduce proof of work. The idea of proof of work is to have a task which is difficult to do and ideally one that has a scalable level of difficulty (if only one machine is working on proof of work it is easy, if many are it can be made more difficult as to keep the rate of difficult problems solved -> bitcoins mined, constant). 
We take the last task of changing one of the entries and try to determine how many spaces to add to the end, so we can get a valid blockchain again

In [ ]:
%%time
from itertools import product
from string import ascii_letters

goal_hash = signed_transactions[1][2]
last_signature = simple_hash_func(signed_transactions[0][0]+signed_transactions[0][1]+'')

for c_vals in product(*[' '+ascii_letters]*6):
    start_text = 'Joel pays Kevin $15. {}'.format(''.join(c_vals))
    cur_hash = simple_hash_func(signed_transactions[1][0]+start_text+last_signature) 
    if cur_hash == goal_hash:
        print('magic text found:', start_text)
        break

In [ ]:
transactions[1][1] = start_text
last_signature = ''
new_signed_transactions = []
for date, content in transactions:
    last_signature = simple_hash_func(date+content+last_signature)
    new_signed_transactions += [[date, content, last_signature]]
                       
for (date, content, new_signature), (_, _, old_signature) in zip(new_signed_transactions, signed_transactions):
    print('\t'.join([date, content]))
    print('\t\t\tnew: ', new_signature)
    print('\t\t\told: ', old_signature)

* We see that by adding characters to the end we can now get the blockchain to be valid again. 

# Task Proof of Work
For this concrete task we have the goal to find a number of multiply the original number by such that the hash we generate has the last character be zero (we can make this a scalable task by saying the last _n_ characters should be 0).

In [ ]:
%%time
my_favorite_number = 12345
proof_of_work = 0  # We don't know what it is yet...
while simple_hash_func(my_favorite_number*proof_of_work)[-1] != "0":
    proof_of_work += 1
print('The solution is proof of work = {}'.format(proof_of_work))
print('Verification',simple_hash_func(my_favorite_number*proof_of_work))

That was clearly not a difficult enough problem and so we need to make it more challenging so it takes longer than the original steps above (it does in fact take longer, but so does printing and writing output to the browser). So now we try having the last 5 digits end in 0

In [ ]:
def calculate_proof_of_work(in_number, out_zeros):
    proof_of_work = 0  # We don't know what it is yet...
    goal_zero_str = ''.join(["0"]*out_zeros)
    while simple_hash_func(my_favorite_number*proof_of_work)[-out_zeros:] != goal_zero_str:
        proof_of_work += 1
    return proof_of_work

In [ ]:
%%time
proof_of_work = calculate_proof_of_work(my_favorite_number, 5)
print('The solution is proof of work = {}'.format(proof_of_work))
print('Verification',simple_hash_func(my_favorite_number*proof_of_work))

# Review
We have now created a proof of work function for taking an input and finding out what it needs to be multiplied by in order to have the desired number of zeros. We can also see how this problem becomes more and more difficult the more zeros it needs to have with examples with 1 zero taking 500$\mu$s and 5 zeros taking 2800000$\mu$s

In [ ]:
%%time
from time import clock
zero_out, time_out = [], []
for i in range(1, 6+1):
    start = clock()
    # average over 3 different numbers
    for j in range(3):
        _ = calculate_proof_of_work(my_favorite_number+j, i)
    t_out = (clock()-start)/3*1000
    time_out += [t_out]
    zero_out += [i]
    print(i,'\t', '%4e ms' % t_out)

Here we plot the results if matplotlib is available otherwise we can just examine the table in more detail

In [ ]:
%matplotlib inline
try:
    import matplotlib.pyplot as plt
    fig, out_ax = plt.subplots(1,1, figsize = (8, 5))
    out_ax.semilogy(zero_out, time_out, 'r+-.')
    out_ax.set_title('Proof of Work Time vs Zeros')
    out_ax.set_ylabel('Proof of Work Time (ms)')
    out_ax.set_xlabel('Number of 0s at end')
except ImportError as ie:
    print('Matplotlib is required to show plots')